In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit




In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def decision_tree_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    max_depth=None,
    min_samples_split=2,
    random_state=42,
    use_cv=False,
    cv_params=None,
    cv=5
):
    """
    Decision Tree regression for time series forecasting with lag features.
    Includes tree visualization and proper time-series cross-validation.
    """
    # 1. Directory structure
    os.makedirs('DecisionTreeRegression', exist_ok=True)
    
    # 2. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 3. Feature Engineering - Lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 4. Time-based split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 5. Model Training (with time-series CV)
    if use_cv:
        param_grid = cv_params or {
            'max_depth': [None, 3, 5, 7, 10],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        tscv = TimeSeriesSplit(n_splits=cv)
        grid_search = GridSearchCV(
            DecisionTreeRegressor(random_state=random_state),
            param_grid,
            cv=tscv,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        model = DecisionTreeRegressor(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=random_state
        )
        model.fit(X_train, y_train)
        best_params = {
            'max_depth': max_depth,
            'min_samples_split': min_samples_split
        }

    # 6. Forecasting
    pred_test = model.predict(X_test)

    # 7. Inverse differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 8. Forecast DataFrame
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test
    })

    # 9. Save results
    forecast_csv_path = os.path.join('DecisionTreeRegression', 'dt_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )

    # 10. Metrics calculation
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    r_squared = model.score(X_train, y_train)
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': r_squared,
        'best_params': best_params,
        'differencing': d,
        'tree_depth': model.get_depth()
    }
    metrics_df = pd.DataFrame([metrics_data])

    metrics_csv_path = os.path.join('DecisionTreeRegression', 'dt_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 11. Visualization
    plt.figure(figsize=(18, 10))
    plot_tree(
        model,
        feature_names=X_train.columns.tolist(),
        filled=True,
        rounded=True,
        proportion=True,
        max_depth=3  # Show first 3 levels for readability
    )
    plt.savefig(os.path.join('DecisionTreeRegression', f'dt_structure_{district_name}.png'))
    plt.close()

    # 12. Forecast plot
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast'], label='DT Forecast', color='purple')
    plt.title(f'Decision Tree Forecast for {district_name}\nRMSE: {rmse:.2f}, Depth: {model.get_depth()}')
    plt.legend()
    plt.savefig(os.path.join('DecisionTreeRegression', f'dt_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'tree_rules': model.tree_,
        'feature_importances': model.feature_importances_
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I55"].asfreq('MS')
        
        results = decision_tree_regression_lags_only(
            ts, 
            district, 
            max_lags=4,
            max_depth=5,
            min_samples_split=10
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district        rmse  r_squared  \
0  AHMEDNAGAR  720.184856     0.8973   

                                 best_params  differencing  tree_depth  
0  {'max_depth': 5, 'min_samples_split': 10}             0           5  

=== Forecast Data ===
     district       date  actual     forecast
0  AHMEDNAGAR 2020-07-01  6225.0  6288.916667
1  AHMEDNAGAR 2020-08-01  6609.0  6288.916667
2  AHMEDNAGAR 2020-09-01  7268.0  6288.916667
3  AHMEDNAGAR 2020-10-01  6799.0  6288.916667
4  AHMEDNAGAR 2020-11-01  5248.0  6288.916667
=== Metrics ===
  district       rmse  r_squared                                best_params  \
0    AKOLA  802.94815    0.76326  {'max_depth': 5, 'min_samples_split': 10}   

   differencing  tree_depth  
0             1           5  

=== Forecast Data ===
  district       date  actual  forecast
0    AKOLA 2020-07-01    1999  1967.500
1    AKOLA 2020-08-01    1927  2563.000
2    AKOLA 2020-09-01    2604  2641.500
3    AKOLA 2020-10-01    2342  2315.875


{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date  actual  forecast
 0  MUMBAI SUBURBAN 2021-02-01    3958    5374.2
 1  MUMBAI SUBURBAN 2021-03-01    4827    5479.4,
 'metrics_df':           district         rmse  r_squared  \
 0  MUMBAI SUBURBAN  1102.553445        0.0   
 
                                  best_params  differencing  tree_depth  
 0  {'max_depth': 5, 'min_samples_split': 10}             1           0  ,
 'tree_rules': <sklearn.tree._tree.Tree at 0x20be46e24f0>,
 'feature_importances': array([0., 0., 0., 0.])}